In [1]:
from IPython.core.debugger import set_trace
from torchtext.datasets import WikiText2, IMDB
import spacy
import re
import html
from torchtext import data, datasets
from spacy.symbols import ORTH
import torch
import torch.nn as nn
import torch.nn.functional as V
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
! ls data/

dogscats  dogscats.zip	wikitext-103  wikitext-2  wikitext-2-v1.zip


In [3]:
spacy_en  = spacy.load('en')

In [4]:
def tokenizer(x):
    return [tok.text for tok in spacy_en.tokenizer(x)]

In [5]:
TEXT = data.Field(lower=True, tokenize = tokenizer)


In [6]:
train, valid, test = WikiText2.splits(TEXT)

In [7]:
TEXT.build_vocab(train, vectors = "fasttext.en.300d")

In [8]:
train_iter, valid_iter, test_iter = data.BPTTIterator.splits(
    (train, valid, test),
    batch_size=16,
    bptt_len=30, # this is where we specify the sequence length
    device = "cuda",
    repeat=False)

In [9]:
class LanguageModel(nn.Module):
    
    def __init__(self, ntoken, ninp,
                 nhid, nlayers, bsz,
                 dropout=0.5):
        super(LanguageModel, self).__init__()
        self.nhid, self.nlayers, self.bsz = nhid, nlayers, bsz
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid,ntoken)

        self.init_weights()
        self.hidden = self.init_hidden(bsz)
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)
 
    def forward(self, input):
        emb = self.drop(self.encoder(input))
        output, self.hidden = self.rnn(emb, self.hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1))
 
    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return (torch.tensor(weight.new(self.nlayers, bsz, self.nhid).zero_().cuda()),
                torch.tensor(weight.new(self.nlayers, bsz, self.nhid).zero_()).cuda())
  
    def reset_history(self):
        self.hidden = tuple(torch.tensor(h.data) for h in self.hidden)


In [10]:
weight_matrix = TEXT.vocab.vectors
model = LanguageModel(weight_matrix.size(0),
weight_matrix.size(1), 200, 3, 16)
model.encoder.weight.data.copy_(weight_matrix)
model.cuda()

LanguageModel(
  (drop): Dropout(p=0.5)
  (encoder): Embedding(28870, 300)
  (rnn): LSTM(300, 200, num_layers=3, dropout=0.5)
  (decoder): Linear(in_features=200, out_features=28870, bias=True)
)

In [ ]:
model.load_state_dict(torch.load('lstm_8.pt'))

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3, betas=(0.7,0.99))
n_tokens = weight_matrix.size(0)

In [12]:
from tqdm import tqdm_notebook as tqdm

def train_epoch(epoch):
    epoch_loss = 0
    for batch in tqdm(train_iter):
        model.reset_history()
    
        optimizer.zero_grad()
        
        text, targets = batch.text, batch.target
        prediction = model(text)
        loss = criterion(prediction.view(-1, n_tokens), targets.view(-1))
        loss.backward()
        
        optimizer.step()
        #set_trace()
        batch_loss = loss.item() * prediction.size(0) * prediction.size(1)
        
        batch_loss /= len(train.examples[0].text)
        
        epoch_loss += batch_loss
    
    val_loss = 0
        
    for index, batch in enumerate(tqdm(valid_iter)):
        model.reset_history()
        text, targets = batch.text, batch.target
        prediction = model(text)
        loss = criterion(prediction.view(-1, n_tokens), targets.view(-1))
        batch_loss = loss.item() * prediction.size(0) * prediction.size(1)
        batch_loss /= len(valid.examples[0].text) 
        
        val_loss += batch_loss
        
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch, epoch_loss, val_loss))
    return val_loss
        

In [ ]:
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=0)
for i in range(30):
    val_losses = []
    val_loss = train_epoch(i)
    val_losses.append(val_loss)
    scheduler.step(val_loss)
    if val_loss == min(val_losses):
        save_path = 'lstm_' + str(i) +'.pt'
        torch.save(model.state_dict(), save_path)

In [13]:
class ClassifierModel(nn.Module):
    
    def __init__(self, ntoken, ninp,
                 nhid, nlayers, bsz, noutputs,
                 dropout=0.5):
        super(ClassifierModel, self).__init__()
        self.nhid, self.nlayers, self.bsz = nhid, nlayers, bsz
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.linear = nn.Linear(nhid,noutputs)

    def forward(self, input):
        bsz = input.size()[1]
        if bsz != self.bsz:
            self.bsz = bsz
        emb = self.drop(self.encoder(input))
        self.hidden = (torch.tensor(emb.data.new(*(self.nlayers, self.bsz, self.nhid)).zero_()),
                       torch.tensor(emb.data.new(*(self.nlayers, self.bsz, self.nhid)).zero_()))
        output, _ = self.rnn(emb, self.hidden)
        return self.linear(output[-1])


In [14]:
model2 = ClassifierModel(weight_matrix.size(0),
weight_matrix.size(1), 200, 3, 4,2)
model2.cuda()

ClassifierModel(
  (drop): Dropout(p=0.5)
  (encoder): Embedding(28870, 300)
  (rnn): LSTM(300, 200, num_layers=3, dropout=0.5)
  (linear): Linear(in_features=200, out_features=2, bias=True)
)

In [15]:
model1_state = model.state_dict()
model2_state = model2.state_dict()
pretrained_state = { k:v for k,v in model1_state.items() if k in model2_state and v.size() == model2_state[k].size() }
model2_state.update(pretrained_state)
model2.load_state_dict(model2_state)

In [16]:
LABEL = data.LabelField(tokenize='spacy')
train2, test2 = datasets.IMDB.splits(TEXT,LABEL)

In [17]:
train_iter2, test_iter2 = data.BucketIterator.splits((train2, test2), device='cuda', batch_size=4,shuffle=True)
LABEL.build_vocab(train2)

In [18]:
optimizer = optim.Adam(model2.parameters(),lr=1e-3)
loss_function = nn.CrossEntropyLoss()
loss_function = loss_function.cuda()
def fit(epoch,model2,data_loader,phase='training'):
    if phase == 'training':
        model2.train()
    if phase == 'validation':
        model2.eval()
    running_loss = 0.0
    running_correct = 0
    run_total = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model2(text)
        output = output.squeeze(1)
        loss = loss_function(output,target)
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += (preds.squeeze() == target).float().sum()
        run_total += len(target)
        running_loss += loss.detach()
        if phase == 'training':            
            loss.backward()
            optimizer.step()
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * float(running_correct)/float(run_total)
    print('Epoch: {}, Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch, loss, accuracy))
    return loss,accuracy
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

In [19]:
from tqdm import tqdm_notebook as tqdm
for epoch in tqdm(range(30)):
    epoch_loss, epoch_accuracy = fit(epoch,model2,train_iter2,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model2,test_iter2,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

HBox(children=(IntProgress(value=0, max=30), HTML(value='')))

Epoch: 0, Loss: 0.1748, Accuracy: 49.6480
Epoch: 0, Loss: 0.1733, Accuracy: 50.0000
Epoch: 1, Loss: 0.1744, Accuracy: 50.2480
Epoch: 1, Loss: 0.1738, Accuracy: 50.0000
Epoch: 2, Loss: 0.1742, Accuracy: 50.2120
Epoch: 2, Loss: 0.1734, Accuracy: 50.0000
Epoch: 3, Loss: 0.1742, Accuracy: 49.9640
Epoch: 3, Loss: 0.1735, Accuracy: 50.0000
Epoch: 4, Loss: 0.1742, Accuracy: 49.5280
Epoch: 4, Loss: 0.1742, Accuracy: 50.0000
Epoch: 5, Loss: 0.1740, Accuracy: 50.1320
Epoch: 5, Loss: 0.1744, Accuracy: 50.0000
Epoch: 6, Loss: 0.1739, Accuracy: 50.5240
Epoch: 6, Loss: 0.1750, Accuracy: 50.0000
Epoch: 7, Loss: 0.1740, Accuracy: 49.7440
Epoch: 7, Loss: 0.1733, Accuracy: 50.0000
Epoch: 8, Loss: 0.1739, Accuracy: 49.5320
Epoch: 8, Loss: 0.1733, Accuracy: 50.0000
Epoch: 9, Loss: 0.1739, Accuracy: 49.7720
Epoch: 9, Loss: 0.1742, Accuracy: 50.0000
Epoch: 10, Loss: 0.1739, Accuracy: 50.3720
Epoch: 10, Loss: 0.1733, Accuracy: 50.0000
Epoch: 11, Loss: 0.1738, Accuracy: 49.6000
Epoch: 11, Loss: 0.1736, Accura

In [20]:
print("train_losses")
print(train_losses)
print("train_accuracy")
print(train_accuracy)
print("val_losses")
print(val_losses)
print("val_accuracy")
print(val_accuracy)

train_losses
[tensor(0.1748, device='cuda:0'), tensor(0.1744, device='cuda:0'), tensor(0.1742, device='cuda:0'), tensor(0.1742, device='cuda:0'), tensor(0.1742, device='cuda:0'), tensor(0.1740, device='cuda:0'), tensor(0.1739, device='cuda:0'), tensor(0.1740, device='cuda:0'), tensor(0.1739, device='cuda:0'), tensor(0.1739, device='cuda:0'), tensor(0.1739, device='cuda:0'), tensor(0.1738, device='cuda:0'), tensor(0.1738, device='cuda:0'), tensor(0.1738, device='cuda:0'), tensor(0.1739, device='cuda:0'), tensor(0.1738, device='cuda:0'), tensor(0.1738, device='cuda:0'), tensor(0.1738, device='cuda:0'), tensor(0.1739, device='cuda:0'), tensor(0.1738, device='cuda:0'), tensor(0.1738, device='cuda:0'), tensor(0.1737, device='cuda:0'), tensor(0.1737, device='cuda:0'), tensor(0.1736, device='cuda:0'), tensor(0.1737, device='cuda:0'), tensor(0.1737, device='cuda:0'), tensor(0.1736, device='cuda:0'), tensor(0.1736, device='cuda:0'), tensor(0.1736, device='cuda:0'), tensor(0.1737, device='cuda:0

In [22]:
for p in model.parameters():
    print(p)

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0653, -0.0930, -0.0176,  ...,  0.1664, -0.1308,  0.0354],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0703, -0.1247, -0.2370,  ...,  0.1170, -0.0455,  0.1239]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[-0.0284, -0.0405, -0.0608,  ..., -0.0057, -0.0217, -0.0378],
        [ 0.0334, -0.0247,  0.0382,  ...,  0.0177,  0.0546,  0.0211],
        [-0.0496,  0.0235, -0.0351,  ..., -0.0190,  0.0028,  0.0683],
        ...,
        [-0.0429,  0.0583, -0.0559,  ...,  0.0653,  0.0500,  0.0285],
        [ 0.0372,  0.0490,  0.0424,  ...,  0.0239, -0.0578, -0.0526],
        [ 0.0456, -0.0687,  0.0115,  ..., -0.0327,  0.0559, -0.0433]],
       device='cuda:0', requires_grad=True)
